EPL: league 39

In [38]:
import requests
import json
import os

# load the environment variables
from dotenv import load_dotenv
load_dotenv()

# Bring your packages onto the path
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# import functions from src/mlb_scores.py
from epl_scores import get_epl_scores, get_epl_odds, get_correct_upsets, create_tweet


# Get Scores from EPL on date

In [39]:
date = '2022-08-14'

In [40]:
score_match_dict = get_epl_scores(date)
score_match_dict

{867963: {'home_team': 'Nottingham Forest',
  'away_team': 'West Ham',
  'home_score': 1,
  'away_score': 0,
  'winner': 'Home'},
 867960: {'home_team': 'Chelsea',
  'away_team': 'Tottenham',
  'home_score': 2,
  'away_score': 2,
  'winner': 'Draw'}}

# Get the "Odds of Winning" for each game on the date

In [41]:
odds_match_dict = get_epl_odds(date)
odds_match_dict

{867960: 'Home', 867963: 'Away'}

# Print Outputs

In [42]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []
for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

print('#EPL Scores {date}'.format(date=date))
if len(upset_ids)>0:
    print("SURPRISES:")
    for id in upset_ids:
        score_match_dict[id]
        print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))
if len(correct_ids)>0:
    print("\nEXPECTED:")
    for id in correct_ids:
        score_match_dict[id]
        print(str("{home_team}: {home_score} - {away_team}: {away_score}").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))

#EPL Scores 2022-08-14
SURPRISES:
Chelsea: 2 - Tottenham: 2
Nottingham Forest: 1 - West Ham: 0
